In [1]:
import os
# please use your key
os.environ["SERPAPI_API_KEY"] = ""

from langchain.agents import initialize_agent, Tool
from langchain.llms import AI21
from langchain.llms.base import LLM
from langchain import LLMMathChain, SerpAPIWrapper


from langchain.llms import OpenAIChat
from langchain.agents import tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from typing import Optional, List, Mapping, Any
import requests
from langchain.llms.utils import enforce_stop_tokens

class VicunaLLM(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        data = {"query": prompt}
        response = requests.post(
            "http://8.130.118.207:8080/post", # this is the IP of my gpu machine
            json= data
        )
        response.raise_for_status()
        output = response.text.strip()[len(prompt):]
        if stop:
            output = enforce_stop_tokens(output, stop)
        return output

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {

        }
llm = VicunaLLM()
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]
llm = VicunaLLM()
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(tools, llm, agent= "zero-shot-react-description", verbose=True, memory=memory,max_iterations=4,early_stopping_method="generate")
agent.run("The current age of the President of the United States multiplied by 0.5.")


/home/pengqu/langchain/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new AgentExecutor chain...
 I need to find the age of the President and then multiply it by 0.5
Action: Search
Action Input: Who is the President of the United States
Observation: Joe Biden
Thought: I need to find Joe Biden's age
Action: Search
Action Input: Joe Biden's age
Observation: 80 years
Thought: I need to multiply 80 by 0.5
Action: Calculator
Action Input: 80 * 0.5

> Entering new LLMMathChain chain...
80 * 0.5```text
80 * 0.5
```
...numexpr.evaluate("80 * 0.5")...

Answer: 40.0
> Finished chain.

Observation: Answer: 40.0
Thought: I now know the final answer
Final Answer: 40.0

> Finished chain.


'40.0'